In [36]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pymysql

# com_list = {'하이닉스' : '000660',
#            '현대차' : '005380',
#            '삼성전자' : '005930',
#            'LG화학' : '051910', '셀트리온' : '068270'}

media_list = {'아시아경제_' : 'asia_',
             '매일경제_': 'maeil_'
             }

for m,j in media_list.items():
    db = pymysql.connect(
        user='root', 
        passwd='1234', 
        host='3.35.70.166', 
        db='proj', 
        charset = 'utf8'
    )

    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = "select * from "+ j +"news_craw_068270 where length(date) = 10"  # date 널값 제외
    cursor.execute(sql)

    result = cursor.fetchall()
    # 데이터 프레임으로 변경
    globals() ['df_'+ j] = pd.DataFrame(result)
    
db.close()


In [ ]:
cel_new = pd.concat([df_asia_, df_maeil_])

In [ ]:
cel_new['year'] = cel_new['date'].str[:4]

In [ ]:
cel_new_2021 = cel_new[cel_new['year'] == '2021'].dropna()

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
cel_new_2021['text'] = cel_new_2021['text'].str.replace('[^가-힣]','')

In [ ]:
dir(okt)

In [ ]:
cel_new_2021['text'].apply(lambda x : okt.nouns(x))

## === 셀트리온 사전 정확도 파악 ===

In [3]:
cel_2018p = pd.read_csv('pos_celltrion_2018.csv')
cel_2019p = pd.read_csv('pos_celltrion_2019.csv')
cel_2020p = pd.read_csv('pos_celltrion_2020.csv')
# cel_2021p = pd.read_csv('pro_celltrion_2021.csv')  2021년를 테스트로 사용

In [4]:
cel_pro = pd.concat([cel_2018p, cel_2019p, cel_2020p])

In [5]:
cel_2018 = pd.read_csv('neg_celltrion_2018.csv')
cel_2019 = pd.read_csv('neg_celltrion_2019.csv')
cel_2020 = pd.read_csv('neg_celltrion_2020.csv')
# cel_2021 = pd.read_csv('neg_celltrion_2021.csv')  2021년를 테스트로 사용

In [6]:
cel_neg = pd.concat([cel_2018, cel_2019, cel_2020])

In [7]:
# 0.6
cel_neg6 = cel_neg[cel_neg['cosine_similarity'] > 0.6]
cel_pro6 = cel_pro[cel_pro['cosine_similarity'] > 0.6]

In [8]:
cel_neg_list = list(cel_neg6['news_word'].values)
cel_pro_list = list(cel_pro6['news_word'].values)

In [9]:
cel_neg_list = list(set(cel_neg_list))
cel_pro_list = list(set(cel_pro_list))

In [10]:
len(cel_neg_list), len(cel_pro_list)

(21, 77)

In [11]:
cel_pro_list

['조정시기',
 '메디콕스메디콕스',
 '퀀텀스탬프',
 '트윈데믹',
 '티플랙스티플랙스',
 '삼륭물산삼륭물산',
 '자이렘',
 '단점',
 '셀루메드셀루메드',
 '레그단비맙',
 '밑돌',
 '스포츠투나잇',
 '중앙디앤엠',
 '본격',
 '휴톡스',
 '문제',
 '개선',
 '쎄미켐동진쎄미켐',
 '증가',
 '재택명령',
 '중거리핵전력조약',
 '추진',
 '약세',
 '것이므',
 '이뮤노메딕스',
 '메디프론메디프론',
 '상쇄하',
 '밑돈',
 '휴메딕스휴메딕스',
 '안트로젠안트로젠',
 '테믹시스',
 '잇츠한불잇츠한불',
 '최선',
 '웃돌',
 '리바운딩',
 '엔투텍엔투텍',
 '스제약위더스',
 '텍드림텍',
 '텍사트인스',
 '실패',
 '주요한',
 '충족',
 '로슈사',
 '프럼파스트프',
 '티플랙스',
 '롱숏펀드',
 '권평오',
 '이노톡스',
 '밑돈다',
 '세릍리온',
 '씽크빅웅진씽크빅',
 '뷰웍스뷰웍스',
 '스크리닝하',
 '젬백스젬백스',
 '침체',
 '불황',
 '특징',
 '리레이팅',
 '추동훈',
 '적극',
 '앰블린파트너스',
 '탭앤고',
 '홍석균',
 '장점',
 '듀피젠트',
 '감소',
 '토픽스지수',
 '엠아이텍',
 '엠에스',
 '증진',
 '텔레칩스텔레',
 '얼머스인베트먼트',
 '판매승인',
 '텍오스코텍',
 '활황',
 '향상',
 '콘엘비세미콘']

In [12]:
cel_neg_list

['하락',
 '급락',
 '호황',
 '결함',
 '부주의',
 '상승',
 '소란',
 '의혹',
 '성공',
 '보합',
 '급등',
 '위축',
 '긍적적',
 '지연',
 '침체',
 '불황',
 '의심',
 '조처',
 '정체',
 '조치',
 '약화']

In [13]:
with open('KOSELF_neg.txt', 'r') as f:
    ko_neg = f.read()
ko_neg = ko_neg.split('\n')

with open('KOSELF_pos.txt', 'r') as f:
    ko_pro = f.read()
ko_pro = ko_pro.split('\n')

In [14]:
ko_neg += cel_neg_list
ko_pro += cel_pro_list

In [15]:
len(ko_neg), len(ko_pro)

(68, 125)

In [16]:
neg=set(ko_neg) - set(ko_pro)
pos=set(ko_pro) - set(ko_neg)

In [17]:
len(neg), len(pos)

(52, 114)

In [18]:
new_cel = pd.read_csv('2021_celltrion.csv')

In [19]:
new_cel['Tokenization'] = new_cel['Tokenization'].str.split(' ') 

In [22]:
new_cel

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,셀트리온,68270,매일경제,2021010106,[돈터뷰] 슈퍼개미에게 묻는다! 사야하는 주식 안되는 주식은?,http://news.mk.co.kr/newsRead.php?no=433&year=...,\n\n\n '보컬'로 알려진 슈퍼개미 김형준 대표는 매일경제 '돈터뷰'에 출연해...,2021-01-04,6,351500,354500,343000,347500,2006483,-0.032033,-1,0,"[보컬, 슈퍼, 개미, 김형준, 대표, 매일경제, 돈터뷰, 출연, 자신, 투자, 철..."
1,셀트리온,68270,매일경제,2021010106,[돈터뷰] 슈퍼개미에게 묻는다! 사야하는 주식 안되는 주식은?,http://news.mk.co.kr/newsRead.php?no=433&year=...,\n\n\n '보컬'로 알려진 슈퍼개미 김형준 대표는 매일경제 '돈터뷰'에 출연해...,2021-01-04,6,351500,354500,343000,347500,2006483,-0.032033,-1,0,"[보컬, 슈퍼, 개미, 김형준, 대표, 매일경제, 돈터뷰, 출연, 자신, 투자, 철..."
2,셀트리온,68270,매일경제,2021010317,[GS칼텍스배 프로기전] 자비로운 손은 없다,http://news.mk.co.kr/newsRead.php?no=4182&year...,\n\n\n 2020~2021 KB국민은행 바둑리그가 6회전을 넘었다. 간판에 걸린...,2021-01-04,17,351500,354500,343000,347500,2006483,-0.032033,-1,0,"[국민은행, 바둑, 리그, 간판, 이름, 마지막, 첫날, 다리, 마지막, 킥스, 연..."
3,셀트리온,68270,매일경제,2021010317,팬데믹 이긴 테슬라…50만대 사상 최대 판매 `질주`,http://news.mk.co.kr/newsRead.php?no=4275&year...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 일론 머스크 테슬라 최...,2021-01-04,17,351500,354500,343000,347500,2006483,-0.032033,-1,0,"[여기, 크게, 머스크, 테슬라, 최고경영자, 지난해, 최고, 작년, 주가, 승률,..."
4,셀트리온,68270,매일경제,2021010317,팬데믹 이긴 테슬라…50만대 사상 최대 판매 `질주`,http://news.mk.co.kr/newsRead.php?no=4275&year...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 일론 머스크 테슬라 최...,2021-01-04,17,351500,354500,343000,347500,2006483,-0.032033,-1,0,"[여기, 크게, 머스크, 테슬라, 최고경영자, 지난해, 최고, 작년, 주가, 승률,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,셀트리온,68270,아시아경제,2021091310,개인 '나홀로 매도'에 코스피 소폭 하락…추석 앞둔 관망세,https://view.asiae.co.kr/article/2021091310530...,[이미지출처=연합뉴스]썝蹂몃낫湲 븘씠肄[아시아경제 지연진 기자]국내 증시가 13일 ...,2021-09-13,10,266500,267000,260500,262000,505510,-0.020561,-1,0,"[이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아, 경제, 연진, 기자, 국내, 증..."
1295,셀트리온,68270,아시아경제,2021091314,경기 둔화 우려 부각…코스닥 1% 넘게 하락,https://view.asiae.co.kr/article/2021091314075...,"코스피 판 개인, 코스닥서 3000억원 넘게 순매수 [이미지출처=연합뉴스]썝蹂몃낫湲...",2021-09-13,14,266500,267000,260500,262000,505510,-0.020561,-1,0,"[코스피, 개인, 코스닥, 매수, 이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아,..."
1296,셀트리온,68270,아시아경제,2021091316,"관망세 짙은 증시, 혼조 마감…'기관 픽' 코스피 오르고, '개인 픽' 코스닥은 하...",https://view.asiae.co.kr/article/2021091316042...,[이미지출처=연합뉴스]썝蹂몃낫湲 븘씠肄[아시아경제 지연진 기자] ] 13일 국내 증...,2021-09-14,16,261500,265000,260500,263000,416684,0.003817,1,0,"[이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아, 경제, 연진, 기자, 국내, 증..."
1297,셀트리온,68270,아시아경제,2021091409,강보합 출발 코스피…3150 근접,https://view.asiae.co.kr/article/2021091409334...,코스닥도 1033대로 올라서 코스피가 소폭 상승세로 장을 출발한 14일 오전 서울 ...,2021-09-14,9,261500,265000,260500,263000,416684,0.003817,1,0,"[코스닥, 대로, 코스피, 상승세, 출발, 오전, 서울, 중구, 하나은행, 딜링룸,..."


In [27]:
score = new_cel[['st_n','st_cd','news','date','Tokenization','UpDown']]

In [28]:
score['dic'] = 0
score['dic_pos'] = 0
score['dic_neg'] = 0
score['pos_per'] = 0.0
score['neg_per'] = 0.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
for i in range(len(score)):
    for j in score['Tokenization'].iloc[i]:
        if j in neg:
            score['dic'].iloc[i] -= 1
            score['dic_neg'].iloc[i] += 1
        if j in pos:
            score['dic'].iloc[i] += 1
            score['dic_pos'].iloc[i] += 1
    score['pos_per'].iloc[i] = score['dic_pos'].iloc[i] / (score['dic_neg'].iloc[i] + score['dic_pos'].iloc[i])
    score['neg_per'].iloc[i] = score['dic_neg'].iloc[i] / (score['dic_neg'].iloc[i] + score['dic_pos'].iloc[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launche

In [32]:
score

,st_n,st_cd,news,date,Tokenization,UpDown,dic,dic_pos,dic_neg,pos_per,neg_per
0,셀트리온,68270,매일경제,2021-01-04,"[보컬, 슈퍼, 개미, 김형준, 대표, 매일경제, 돈터뷰, 출연, 자신, 투자, 철...",-1,-9,12,21,0.363636,0.636364
1,셀트리온,68270,매일경제,2021-01-04,"[보컬, 슈퍼, 개미, 김형준, 대표, 매일경제, 돈터뷰, 출연, 자신, 투자, 철...",-1,-9,12,21,0.363636,0.636364
2,셀트리온,68270,매일경제,2021-01-04,"[국민은행, 바둑, 리그, 간판, 이름, 마지막, 첫날, 다리, 마지막, 킥스, 연...",-1,0,0,0,NaN,NaN
3,셀트리온,68270,매일경제,2021-01-04,"[여기, 크게, 머스크, 테슬라, 최고경영자, 지난해, 최고, 작년, 주가, 승률,...",-1,-9,9,18,0.333333,0.666667
4,셀트리온,68270,매일경제,2021-01-04,"[여기, 크게, 머스크, 테슬라, 최고경영자, 지난해, 최고, 작년, 주가, 승률,...",-1,-9,9,18,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
1294,셀트리온,68270,아시아경제,2021-09-13,"[이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아, 경제, 연진, 기자, 국내, 증...",-1,-7,3,10,0.230769,0.769231
1295,셀트리온,68270,아시아경제,2021-09-13,"[코스피, 개인, 코스닥, 매수, 이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아,...",-1,-4,11,15,0.423077,0.576923
1296,셀트리온,68270,아시아경제,2021-09-14,"[이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아, 경제, 연진, 기자, 국내, 증...",1,-7,5,12,0.294118,0.705882
1297,셀트리온,68270,아시아경제,2021-09-14,"[코스닥, 대로, 코스피, 상승세, 출발, 오전, 서울, 중구, 하나은행, 딜링룸,...",1,-11,4,15,0.210526,0.789474


In [33]:
(((score.UpDown >= 0) & (score.dic >= 0)).sum() + ((score.UpDown < 0) & (score.dic < 0)).sum())/len(score)

0.5219399538106235

In [34]:
score.UpDown.value_counts(normalize = True)

-1    0.584296
 1    0.415704
Name: UpDown, dtype: float64

In [35]:
score.dic.describe()

count    1299.000000
mean       -9.143957
std        17.309475
min       -74.000000
25%       -20.000000
50%        -4.000000
75%         3.000000
max        42.000000
Name: dic, dtype: float64

In [77]:
(score.pos_per >= 0.9).sum()

261

In [78]:
((score.pos_per >= 0.9) & (score.UpDown >= 0)).sum()

121

In [76]:
((score.pos_per >= 0.9) & (score.UpDown >= 0)).sum() /(score.pos_per >= 0.9).sum()

0.46360153256704983

In [44]:
score.to_csv('cel_2021_score.csv')

## =====하이닉스 사전 정확도 파악 ======

In [39]:
hy_2018p = pd.read_csv('pos_sk_2018.csv')
hy_2019p = pd.read_csv('pos_sk_2019.csv')
hy_2020p = pd.read_csv('pos_sk_2020.csv')
# hy_2021p = pd.read_csv('pos_sk_2021.csv')  2021년를 테스트로 사용

hy_pro = pd.concat([hy_2018p, hy_2019p, hy_2020p])

hy_2018 = pd.read_csv('neg_sk_2018.csv')
hy_2019 = pd.read_csv('neg_sk_2019.csv')
hy_2020 = pd.read_csv('neg_sk_2020.csv')
# hy_2021 = pd.read_csv('neg_sk_2021.csv')  2021년를 테스트로 사용

hy_neg = pd.concat([hy_2018, hy_2019, hy_2020])

In [40]:
# 0.6
hy_neg6 = hy_neg[hy_neg['cosine_similarity'] > 0.6]
hy_pro6 = hy_pro[hy_pro['cosine_similarity'] > 0.6]

hy_neg_list = list(set(hy_neg6['news_word'].values))
hy_pro_list = list(set(hy_pro6['news_word'].values))

In [41]:
len(hy_neg_list), len(hy_pro_list)

(19, 91)

In [42]:
with open('KOSELF_neg.txt', 'r') as f:
    ko_neg = f.read()
ko_neg = ko_neg.split('\n')

with open('KOSELF_pos.txt', 'r') as f:
    ko_pro = f.read()
ko_pro = ko_pro.split('\n')

ko_neg += hy_neg_list
ko_pro += hy_pro_list

# 교집합 제거
neg=set(ko_neg) - set(ko_pro)
pos=set(ko_pro) - set(ko_neg)

In [43]:
new_hy = pd.read_csv('2021_sk.csv')
new_hy['Tokenization'] = new_hy['Tokenization'].str.split(' ')

score_sk = new_hy[['st_n','st_cd','news','date','Tokenization','UpDown']]

score_sk['dic'] = 0
score_sk['dic_pos'] = 0
score_sk['dic_neg'] = 0
score_sk['pos_per'] = 0.0
score_sk['neg_per'] = 0.0

for i in range(len(score_sk)):
    for j in score_sk['Tokenization'].iloc[i]:
        if j in neg:
            score_sk['dic'].iloc[i] -= 1
            score_sk['dic_neg'].iloc[i] += 1
        if j in pos:
            score_sk['dic'].iloc[i] += 1
            score_sk['dic_pos'].iloc[i] += 1
    score_sk['pos_per'].iloc[i] = score_sk['dic_pos'].iloc[i] / (score_sk['dic_neg'].iloc[i] + score_sk['dic_pos'].iloc[i])
    score_sk['neg_per'].iloc[i] = score_sk['dic_neg'].iloc[i] / (score_sk['dic_neg'].iloc[i] + score_sk['dic_pos'].iloc[i])

In [45]:
score_sk

,st_n,st_cd,news,date,Tokenization,UpDown,dic,dic_pos,dic_neg,pos_per,neg_per
0,sk하이닉스,660,매일경제,2021-01-04,"[신년, 기획, 경제, 기상도, 이재용, 삼성, 전자, 부회장, 왼쪽, 지난해, 충...",1,7,7,0,1.000000,0.000000
1,sk하이닉스,660,매일경제,2021-01-04,"[수출, 화물, 컨테이너선, 부산, 출발, 대양, 위기, 기회, 오늘, 한국, 기업...",1,-1,5,6,0.454545,0.545455
2,sk하이닉스,660,매일경제,2021-01-04,"[여기, 크게, 증시, 전문가, 올해, 한국, 증시, 최선호, 종목, 전자, 꼽으면...",1,5,9,4,0.692308,0.307692
3,sk하이닉스,660,매일경제,2021-01-04,"[신년, 기획, 한국, 증시, 코스피, 이후, 허락, 차례, 경신, 돌파, 시도, ...",1,1,2,1,0.666667,0.333333
4,sk하이닉스,660,매일경제,2021-01-05,"[국내, 주요, 대기업, 최고경영자, 신년사, 통해, 새해, 경영, 목표, 코로나,...",1,5,5,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1930,하이닉스,660,아시아경제,2021-09-15,"[이미지, 출처, 연합뉴스, 몃낫, 븘씠, 아시아, 경제, 이민지, 기자, 코스피,...",-1,-30,9,39,0.187500,0.812500
1931,하이닉스,660,아시아경제,2021-09-15,"[동반성, 위원회, 평가, 결과, 발표, 협력, 대상, 공정, 거래, 정착, 상생,...",-1,1,15,14,0.517241,0.482759
1932,하이닉스,660,아시아경제,2021-09-15,"[외국인, 수로, 전환, 부양책, 대감, 증시, 유입, 이미지, 출처, 연합뉴스, ...",-1,-27,1,28,0.034483,0.965517
1933,하이닉스,660,아시아경제,2021-09-15,"[정부, 출연, 연구원, 기술, 창업, 대박, 신화, 속출, 연구원, 일반인, 기업...",-1,5,6,1,0.857143,0.142857


In [93]:
sss = score_sk[(score_sk.pos_per != 1) & (score_sk.neg_per != 1)]

In [98]:
(sss.pos_per == 0).sum()

0

In [94]:
(sss.pos_per == 1).sum()

0

In [95]:
((sss.pos_per >= 0.9) & (sss.UpDown >= 0)).sum()

24

In [96]:
(sss.pos_per >= 0.9).sum()

45

In [113]:
sss[((sss.pos_per >= 0.7) & (sss.UpDown < 0))]

,st_n,st_cd,news,date,Tokenization,UpDown,dic,dic_pos,dic_neg,pos_per,neg_per
37,sk하이닉스,660,매일경제,2021-01-15,"[여기, 크게, 하이, 투자, 증권, 한국판, 뉴딜, 정책, 구조, 성장, 디지털,...",-1,2,3,1,0.750000,0.250000
55,sk하이닉스,660,매일경제,2021-01-22,"[사진, 용인, 굿모닝, 프레스티지, 여기, 크게, 동안, 용인, 하이닉스, 세계,...",-1,5,6,1,0.857143,0.142857
66,sk하이닉스,660,매일경제,2021-01-29,"[하이닉스, 이천, 연구개발, 센터, 사진, 제공, 하이닉스, 하이닉스, 지난해, ...",-1,13,16,3,0.842105,0.157895
97,sk하이닉스,660,매일경제,2021-02-08,"[세대, 성과급, 논란, 산업, 강타, 직원, 제기, 성과급, 문제, 대해, 하이닉...",-1,10,11,1,0.916667,0.083333
113,sk하이닉스,660,매일경제,2021-02-17,"[쿠팡, 미국, 뉴욕, 증권, 거래소, 상장, 나비효과, 국내, 경쟁, 기업인, 네...",-1,9,10,1,0.909091,0.090909
...,...,...,...,...,...,...,...,...,...,...,...
1895,sk하이닉스,660,아시아경제,2021-09-06,"[유해, 가스, 스크, 러버, 장비, 판매, 아시아, 경제, 유현, 기자, 반도체,...",-1,17,19,2,0.904762,0.095238
1897,sk하이닉스,660,아시아경제,2021-09-06,"[한솔, 케미칼, 과산화수소, 프리, 커서, 수요, 증가, 수혜, 소재, 부문, 성...",-1,19,20,1,0.952381,0.047619
1900,sk하이닉스,660,아시아경제,2021-09-07,"[건설, 경기도, 용인시, 일원, 힐스테이트, 용인, 둔전역, 순위, 청약, 결과,...",-1,2,3,1,0.750000,0.250000
1902,sk하이닉스,660,아시아경제,2021-09-07,"[넥실, 리스, 정읍, 공장, 폐기물, 재활용, 다른, 사업, 인증, 확대, 추진,...",-1,7,8,1,0.888889,0.111111


In [111]:
sss[((sss.pos_per >= 0.7) & (sss.UpDown < 0))]['dic_neg'].describe()

count    192.000000
mean       1.687500
std        1.071404
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        7.000000
Name: dic_neg, dtype: float64

In [110]:
sss[((sss.pos_per >= 0.7) & (sss.UpDown < 0))]['dic_pos'].describe()

count    192.000000
mean       7.744792
std        4.598255
min        3.000000
25%        4.000000
50%        6.000000
75%       10.000000
max       27.000000
Name: dic_pos, dtype: float64

In [103]:
((sss.pos_per >= 0.9) & (sss.UpDown >= 0)).sum() / (sss.pos_per >= 0.9).sum()

0.5333333333333333

In [99]:
(((sss.UpDown >= 0) & (sss.dic >= 0)).sum() + ((sss.UpDown < 0) & (sss.dic < 0)).sum())/len(sss)

0.49351535836177474

In [47]:
(((score_sk.UpDown >= 0) & (score_sk.dic >= 0)).sum() + ((score_sk.UpDown < 0) & (score_sk.dic < 0)).sum())/len(score_sk)

0.5023255813953489

In [82]:
score_sk.UpDown.value_counts()

-1    1061
 1     874
Name: UpDown, dtype: int64

In [49]:
score_sk.dic.describe()

count    1935.000000
mean       -2.038243
std         7.672968
min       -37.000000
25%        -7.000000
50%         0.000000
75%         3.000000
max        50.000000
Name: dic, dtype: float64

In [46]:
score_sk.to_csv('./result/sk_2021_score.csv')

In [81]:
len(score_sk)

1935

In [80]:
((score_sk.pos_per == 1) & (score_sk.UpDown < 0)).sum()

110

In [75]:
(score_sk.pos_per == 1).sum() 

206

In [65]:
((score_sk.pos_per >= 0.9) & (score_sk.UpDown >= 0)).sum() /(score_sk.pos_per >= 0.9).sum() 

0.47808764940239046

In [56]:
len(score_sk)

1935